# Tests utilisation de la base de donnée

In [10]:
import numpy as np
import pandas as pd

In [11]:
def convert_attributes_into_pandas(filename):
    return pd.read_csv(filename)

In [12]:
matrix_attributes = convert_attributes_into_pandas("attributes_data.csv")
matrix_attributes.head()

,ID,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
1,000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
2,000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
3,000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
4,000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1


In [13]:
indexes = matrix_attributes.columns
indexes

Index(['ID', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive',
       'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose',
       'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows',
       'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair',
       'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open',
       'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin',
       'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns',
       'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings',
       'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace',
       'Wearing_Necktie', 'Young'],
      dtype='object')

## Réduction matrice

In [42]:
#On souhaite pouvoir réduire la matrice selon certaines colonnes, dont les attributs correspondants seront fixés
#et ne pourront donc pas changer pendant l'algorithme génétique.
#Pour cela on va récupérer une liste d'attributs avec certaines valeurs et utiliser cette liste pour réduire la matrice

fixed_attributes = {"Attractive" : 1 ,"Eyeglasses" : 1 , "Gray_Hair" : -1 , "Brown_Hair" : -1 , "Black_Hair" : 1 ,"Blond_Hair" : -1 , "Pale_Skin":1}

In [43]:
x = matrix_attributes[matrix_attributes.Attractive == 1]

In [51]:
def matrix_reduction(matrix_attributes,fixed_attributes):
    attributes = list(fixed_attributes.keys())
    for i in range(len(attributes)):
        attribute = attributes[i]
        matrix_attributes[matrix_attributes[attribute] != fixed_attributes[attribute]]
    return matrix_attributes


In [55]:
print(fixed_attributes)
test = list(fixed_attributes.keys())
print(test)
atr = test[3]
print(atr)
matrix_attributes[matrix_attributes[atr] != fixed_attributes[test[3]]]
matrix_attributes.Brown_Hair

{'Attractive': 1, 'Eyeglasses': 1, 'Gray_Hair': -1, 'Brown_Hair': -1, 'Black_Hair': 1, 'Blond_Hair': -1, 'Pale_Skin': 1}
['Attractive', 'Eyeglasses', 'Gray_Hair', 'Brown_Hair', 'Black_Hair', 'Blond_Hair', 'Pale_Skin']
Brown_Hair


0         1
1         1
2        -1
3        -1
4        -1
         ..
202594   -1
202595   -1
202596   -1
202597   -1
202598   -1
Name: Brown_Hair, Length: 202599, dtype: int64

In [52]:
matrix_reduction(matrix_attributes,fixed_attributes)

,ID,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
1,000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
2,000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
3,000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
4,000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202594,202595.jpg,-1,-1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1
202595,202596.jpg,-1,-1,-1,-1,-1,1,1,-1,-1,...,-1,1,1,-1,-1,-1,-1,-1,-1,1
202596,202597.jpg,-1,-1,-1,-1,-1,-1,-1,-1,1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
202597,202598.jpg,-1,1,1,-1,-1,-1,1,-1,1,...,-1,1,-1,1,1,-1,1,-1,-1,1
